<a href="https://colab.research.google.com/github/wer2956/Deep_Learning_Team6/blob/master/colabTrial01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os
from glob import glob

np.random.seed(5)

Using TensorFlow backend.


In [0]:
img_dir1 = '/content/drive/My Drive/data_origin/train'
categories1 = os.listdir(img_dir1)
nb_classes1 = len(categories1)

img_dir2 = '/content/drive/My Drive/data_origin/test'
categories2 = os.listdir(img_dir2)
nb_classes2 = len(categories2)

print(categories1,'\n')
print(categories2)

['dyed-resection-margins', 'dyed-lifted-polyps', 'polyps', 'normal-cecum', 'normal-z-line', 'esophagitis', 'normal-pylorus', 'ulcerative-colitis'] 

['normal-z-line', 'dyed-lifted-polyps', 'dyed-resection-margins', 'polyps', 'normal-cecum', 'esophagitis', 'normal-pylorus', 'ulcerative-colitis']


In [0]:
train_dir = os.path.join('/content/drive/My Drive/data_origin/train')
test_dir = os.path.join('/content/drive/My Drive/data_origin/test')

#train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1) # train data로부터 10%를 떼어서 validation data를 만든다
train_datagen = ImageDataGenerator(
      rescale=1./255,                                                     # 영상이 0~1사이의 범위를 가지게 된다(normalization)
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(720,576), # (720, 576)
                                                   batch_size=100,
                                                   class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                 target_size=(720,576),
                                                 batch_size=10,
                                                 class_mode='categorical')


Found 7200 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [0]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

배치 데이터 크기: (100, 720, 576, 3)
배치 레이블 크기: (100, 8)


In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(720,576,3)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, kernel_initializer='he_normal', activation='relu'))
model.add(Dense(64, kernel_initializer='he_normal',activation='relu'))
model.add(Dense(8, activation='softmax'))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 718, 574, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 716, 572, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 358, 286, 64)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6552832)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               838762624 
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 8)                

In [0]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['acc'])

In [0]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [0]:
hist = model.fit_generator(train_generator,
                           steps_per_epoch=200,
                           epochs=10,
                           verbose=1,
                           callbacks=None,
                           validation_data=test_generator,
                           validation_steps=10,
                           class_weight=None,
                           max_queue_size=5)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 63 could not be retrieved. It could be because a worker has died.
  UserWarning)


In [0]:
model.save_weights('first_try.h5')

In [0]:
scores = model.evaluate_generator(test_generator, steps=10)
print(scores)

In [0]:
out = model.predict_generator(test_generator, steps=10)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(out)